# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [ ]:
mongoimport --db posts --collection rows --file establishments.json --jsonArray

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [5]:
# assign the uk_food database to a variable name
db = mongo['uk_food']


In [ ]:
# review the collections in our database
db.establishment.find()

In [6]:
# assign the collection to a variable
establishments = db['establishments']

In [ ]:
db_list = mongo.list_database_names()

In [ ]:
# Print the list of databases
print("Available databases:")
for db in db_list:
    print(db)

In [ ]:
if 'uk_food' in db_list:
    print("'uk_food' database is present.")
else:
    print("'uk_food' database is not present.")

In [ ]:
# Check if 'establishments' collection is in the list
if 'establishments' in db_list:
    print("'establishments' collection is present.")
else:
    print("'establishments' collection is not present.")

In [ ]:
# Find one document from the collection
document =establishments.find_one()

# Display the document using pprint
pprint(document)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our new database
db.establishment.find()

In [ ]:
# review a document in the establishments collection


In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [7]:
# Define the new restaurant entry
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}

# Insert the new restaurant entry into the collection
result = establishments.insert_one(new_restaurant)

# Print the inserted document's ID
print("Inserted document ID:", result.inserted_id)

Inserted document ID: 65f5510af7b73a8e7ce87f2c


In [ ]:
# Create a dictionary for the new restaurant data


In [ ]:
# Insert the new restaurant into the collection


In [ ]:
# Check that the new restaurant was inserted


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [9]:
# Define the query
query = {
    "BusinessType": "Restaurant/Cafe/Canteen"
}
# Define projection to return only the specified fields
projection = {
    "_id": 0,  # Exclude the _id field
    "BusinessTypeID": 1,
    "BusinessType": 1
}

# Execute the query
result = establishments.find_one(query, projection)

# Print the result
print(result)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': ''}


Matched: 1
Modified: 1


In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields


3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Define the filter to find the document to update
filter_query = {
    "BusinessName": "Penang Flavours"
}

# Define the update operation
update_query = {
    "$set": {
        "BusinessTypeID": "your_business_type_id",  # Replace 'your_business_type_id' with the BusinessTypeID you found
        "NewRatingPending": False  # Set NewRatingPending to False since the restaurant has been rated now
    }
}

# Perform the update operation
update_result = establishments.update_one(filter_query, update_query)

# Print the acknowledgment
print("Matched:", update_result.matched_count)
print("Modified:", update_result.modified_count)

In [ ]:
# Update the new restaurant with the correct BusinessTypeID


In [ ]:
# Confirm that the new restaurant was updated


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [11]:
# Step 1: Check how many documents contain the Dover Local Authority
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print("Number of establishments in Dover before deletion:", dover_count)

# Step 2: Remove any establishments within the Dover Local Authority
delete_result = establishments.delete_many({"LocalAuthorityName": "Dover"})
print("Number of establishments deleted from Dover:", delete_result.deleted_count)

# Step 3: Check the number of documents to ensure they were deleted
dover_count_after_deletion = establishments.count_documents({"LocalAuthorityName": "Dover"})
print("Number of establishments in Dover after deletion:", dover_count_after_deletion)

Number of establishments in Dover before deletion: 0
Number of establishments deleted from Dover: 0
Number of establishments in Dover after deletion: 0


In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"


In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"


In [ ]:
# Check if any remaining documents include Dover


In [ ]:
# Check that other documents remain with 'find_one'


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [12]:
# Convert latitude and longitude to decimal numbers
establishments.update_many({}, [
    {"$set": {"geocode.longitude": {"$toDouble": "$geocode.longitude"}}},
    {"$set": {"geocode.latitude": {"$toDouble": "$geocode.latitude"}}}
])

# Convert RatingValue to integer numbers
establishments.update_many({}, [
    {"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}
])

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [ ]:
# Change the data type from String to Decimal for longitude and latitude


Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue


In [ ]:
# Check that the coordinates and rating value are now numbers
